In [1]:
import sys
from pathlib import Path
import os

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
if os.path.exists(f"{root_dir}/.env"):
    settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /Users/zhangman/Documents/25_p2/scalable/lab/mlfs-book
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [2]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [15]:
project = hopsworks.login()
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
all_locations = json.loads(location_str)
location =all_locations["sensor1"]
country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

2025-11-15 13:02:11,169 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-15 13:02:11,186 INFO: Initializing external client
2025-11-15 13:02:11,187 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-15 13:02:12,762 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1296532
3e6104ddc2479da9ec9bf4ef42c6bd008c127592


'{"sensor1": {"country": "unitedkingdom", "city": "london", "street": "london", "aqicn_url": "https://api.waqi.info/feed/@5724", "csv_file": "/data/air-quality-data.csv", "latitude": "51.50735096", "longitude": "-0.1277583"}, "sensor2": {"country": "unitedkingdom", "city": "londonboroughofwandsworth", "street": "nineelmslane", "aqicn_url": "https://api.waqi.info/feed/A516589", "csv_file": "/data/air-quality-data-2.csv", "latitude": "51.50735096", "longitude": "-0.1277583"}, "sensor3": {"country": "unitedkingdom", "city": "londonboroughofwandsworth", "street": "batterseaparkroad", "aqicn_url": "https://api.waqi.info/feed/A516559/", "csv_file": "/data/air-quality-data-3.csv", "latitude": "51.50735096", "longitude": "-0.1277583"}}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [7]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [8]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
aq_lagged_df = air_quality_fg.filter(air_quality_fg.date>=today-datetime.timedelta(days = 4)).show(3)
aq_lagged_df = aq_lagged_df.sort_values('date')

aq_today_df["pm25_lag1"] = aq_lagged_df.iloc[-1]["pm25"]
aq_today_df["pm25_lag2"] = aq_lagged_df.iloc[-2]["pm25"]
aq_today_df["pm25_lag3"] = aq_lagged_df.iloc[-3]["pm25"]
aq_today_df["pm25_lagged"] = aq_today_df[["pm25_lag1", "pm25_lag2", "pm25_lag3"]].mean(axis=1)

aq_today_df

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.68s) 


,pm25,country,city,street,date,url,pm25_lag1,pm25_lag2,pm25_lag3,pm25_lagged
0,60.0,unitedkingdom,londonboroughofwandsworth,nineelmslane,2025-11-14,https://api.waqi.info/feed/A516589,50.0,39.0,41.0,43.333332


In [9]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   pm25         1 non-null      float32       
 1   country      1 non-null      object        
 2   city         1 non-null      object        
 3   street       1 non-null      object        
 4   date         1 non-null      datetime64[ns]
 5   url          1 non-null      object        
 6   pm25_lag1    1 non-null      float32       
 7   pm25_lag2    1 non-null      float32       
 8   pm25_lag3    1 non-null      float32       
 9   pm25_lagged  1 non-null      float32       
dtypes: datetime64[ns](1), float32(5), object(4)
memory usage: 188.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [10]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 51.5°N -0.25°E
Elevation 16.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-14,13.15,1.0,9.449572,72.255241,londonboroughofwandsworth
1,2025-11-15,12.70,0.0,2.595997,326.309906,londonboroughofwandsworth
2,2025-11-16,9.70,0.0,11.384199,18.435053,londonboroughofwandsworth
3,2025-11-17,7.20,0.0,15.042659,338.962402,londonboroughofwandsworth
4,2025-11-18,4.90,0.0,7.704336,217.405426,londonboroughofwandsworth
5,2025-11-19,5.30,0.2,16.870138,309.805511,londonboroughofwandsworth
6,2025-11-20,5.40,0.0,17.026896,346.551361,londonboroughofwandsworth


In [11]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         7 non-null      datetime64[ns]
 1   temperature_2m_mean          7 non-null      float32       
 2   precipitation_sum            7 non-null      float32       
 3   wind_speed_10m_max           7 non-null      float32       
 4   wind_direction_10m_dominant  7 non-null      float32       
 5   city                         7 non-null      object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 352.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [12]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2025-11-14 22:25:41,720 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1296532/fs/1284163/fg/1668849


Uploading Dataframe: 100.00% |█| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time


Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1296532/jobs/named/air_quality_1_offline_fg_materialization/executions


(Job('air_quality_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 739499
         }
       },
       "result": {
         "observed_value": 60.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-14T09:25:41.000719Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectat

In [13]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-14 22:26:21,363 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1296532/fs/1284163/fg/1703282


Uploading Dataframe: 100.00% |█| Rows 7/7 | Elapsed Time: 00:00 | Remaining Time


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1296532/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-14 22:26:38,685 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-14 22:26:41,930 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-14 22:28:27,787 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-14 22:28:27,945 INFO: Waiting for log aggregation to finish.
2025-11-14 22:28:36,525 INFO: Execution finished successfully.


(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 747556
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-14T09:26:21.000362Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
